In [1]:
from math import exp
from scipy.optimize import minimize

# Dit is een functie waarvan ik weet dat ze een maximum heeft voor
# w = [3.2, 1.7], eens kijken of we dat kunnen vinden.
def f(w):
    return exp(-(3.2 - w[0])**2 - (1.7 - w[1])**2)

# Omdat de methode naar een minimum zoekt en wij naar een maximum,
# laat ik naar het minimum van het opposite van f() zoeken.

def opposite(w):
    return -f(w)

start_vector = [2, 3] #[2, 3]

r = minimize(opposite, start_vector)

if r.success:
    print(f'Optimum gevonden voor w = [{r.x[0]}, {r.x[1]}]')
else:
    print('Geen optimum gevonden')

Optimum gevonden voor w = [3.200000368025025, 1.6999995384816942]


In [4]:
__author__ = 'erwich/sikkel'

import os

from sys import argv, stderr

import numpy as np
from scipy.optimize import linear_sum_assignment

def error(*args, **kwargs):
    '''
    Prints error messages.
    '''

    print(*args, file=stderr, **kwargs)
    exit(1)

def parse_ann(annFile):
    '''
    Parses a plain text brat annotation file for a given repository
    specified in `compare_ann()`, and returns an np.array of corefs and singletons
    '''

    errors = 0

    results_list = []
    t2mDict = {}
    singletonSet = set()
    dataPartsList = []

    firstChars = {'T', '#', '*'}
    cClass = 0

    with open(annFile) as fh:
        for (i, line) in enumerate(fh):
            epos = f'{i + 1} '
            line = line.rstrip('\n')
            firstChar = line[0]

            if firstChar not in firstChars:
                error(f'{epos}Unrecognized line "{line}"')
                errors +=1
                continue

            numFields = 2 if firstChar =='*' else 3
            parts = line.split('\t')

            if len(parts) != numFields:
                error(f'{epos}line does not have exactly {numFields} parts: "{line}"')
                errors += 1
                continue

            if firstChar == 'T':
                (tPart, mentionStr, aWord) = parts
                mParts = mentionStr.split()
                if len(mParts) != 3:
                    error(f'{epos}T-line mention does not have exactly 3 parts: "{line}"')
                    errors += 1
                    continue
                t2mDict[tPart] = mentionStr
                singletonSet.add(mentionStr)

            elif firstChar == '*':
                corefSets = set()
                (char, data) = parts
                dataParts = data.split()
                if len(dataParts) <= 1 or dataParts[0] != 'Coreference':
                    error(f'{epos}*-line spec does not have the right parts: "{line}"')
                    errors += 1
                    continue
                cClass += 1
                dataPartsList.append(dataParts)
        
        for l in dataPartsList:
            corefSets = set()
            for tPart in l[1:]:
                if tPart in corefSets:
                    error(f'{epos}*-"{tPart} occurs in multiple classes "{corefSets[tPart]}" in "{line}"')
                    errors += 1
                    continue
                corefSets.add(t2mDict[tPart])
                singletonSet.discard(t2mDict[tPart])
            
            results_list.append(corefSets)
        
        results_list.append(singletonSet) #results_list[-1] is the singletonSet
    
    if errors:
        error(f'There are {errors} errors in annotation file')
    
    results_array = np.array(results_list)

    return results_array

def selection_size(L, s):
    '''
    L is a list of sets, s is a set of indices in L. This function
    returns the total of the cardinalities of the sets selected by s.
    '''
    
    r = 0
    for e in s:
        r += len(L[e])
    return r

def distance(s1, s2):
    return (len(s1 - s2), len(s1 & s2), len(s2 - s1))
    
def match(A, B, d):
    '''
    Matches the nodes in a bipartite graph with n and k nodes using the
    distance function d(i,j) and stores the matching in array r.
    The unpaired corefs are calculated by selection_size().
    '''

    n = len(A)
    k = len(B)
    cost = np.zeros((n, k))
    
    for i in range(n):
        for j in range(k):
            (L, M, R) = distance(A[i], B[j])
            cost[i, j] = (L + R) / (L + M + R)
    row_ind, col_ind = linear_sum_assignment(cost)
    unpaired_A = selection_size(A, set(range(n)) - set(row_ind))
    unpaired_B = selection_size(B, set(range(k)) - set(col_ind))
    total_cost = cost[row_ind, col_ind].sum() + unpaired_A + unpaired_B
    uA = set(range(n)) - set(row_ind)
    uB = set(range(k)) - set(col_ind)
    
    return total_cost.sum(), row_ind, col_ind, uA, uB

def make_r(B):
    '''
    Makes an array of zero's with the length of array B.
    '''
    
    r = np.zeros(len(B), dtype=int)
    return r

def set_distance(sA, sB):
    '''
    Prints the paired and unpaired coref sets of annotator A and B.
    Returns the difference (Left, Right),
    intersection (M), symmetric difference (D) and distance() (d) of two sets. 
    '''
    
    L = len(sA-sB)
    M = len(sA&sB)
    R = len(sB-sA)
    #D = L + R
    #d = D/(L+M+R)
    sum_it(L, M, R)
    return L, M, R

def sum_it(L, M, R):
    '''
    Takes as input a txt file, tab separated.
    Accumulates the coref differences per txt file
    and prints its total on standard output.
    Example call: python3 acc.py Psalms_*.ann
    '''
        
    Lt += L
    Mt += M
    Rt += R
    return Lt, Mt, Rt

def compare_corefs(A, B, uA, rx, cx, uB):
    for i in range(len(rx)):
        l, m, r = set_distance(A[rx[i]], B[cx[i]])
    l1, m1, r1 = sum_it(l, m, r)
    for i in uA:
        L, M, R = set_distance(A[i], set())
    l2, m2, r2 = sum_it(l, m, r)
    for i in uB:
        L, M, R = set_distance(set(), B[i]) 
    l3, m3, r3 = sum_it(l, m, r)
    
    D = l1 + l2 + l3 + r1 + r2 + r3
    d = D/(l1 + m1 + r1 + l2 + m2 + r2 + l3 + m3 + r3)
    return d

def CompareAnn(pathA, pathB):
    '''
    Executes the comparison between the brat annotation files of annotator A and B.
    '''
    
    annFileA = os.path.expanduser(pathA)
    annFileB = os.path.expanduser(pathB)
    
    results_array_A = parse_ann(annFileA)
    results_array_B = parse_ann(annFileB)
    
    make_r(results_array_B)
    cost, rx, cx, uA, uB = match(results_array_A, results_array_B, distance)
    d = compare_corefs(results_array_A, results_array_B, uA, rx, cx, uB)

In [ ]:
if __name__ == "__main__":
    CompareAnn(argv[1], argv[2])

In [8]:
new_ann = os.path.expanduser('~/github/cmerwich/participant-analysis/mimi/mimi-opt/*.ann')
out = '*.ann'
old_file = os.path.expanduser('~/Sites/brat/data/coref/Psalms/annotate/*.txt') # old file .txt
mimi_file = os.path.expanduser('~/github/cmerwich/participant-analysis/mimi/mimi-opt/*.txt') # mimi file .txt

path_a = '~/Sites/brat/data/coref/Psalms/annotate'
path_b = '~/github/cmerwich/participant-analysis/mimi/mimi-opt'

/Users/Christiaan/github/cmerwich/participant-analysis/mimi/mimi-opt


In [1]:
import os
from math import exp
from scipy.optimize import minimize

from functools import lru_cache
from translate import translate

@lru_cache(maxsize=128)
def Optimise(w):
    '''
    w = weight_vector = []
    '''
    
    mimi('Psalms', 105, 105, w)
    new_ann = os.path.expanduser('~/github/cmerwich/participant-analysis/mimi/mimi-opt/Psalms_105.ann') # mimi file.ann
    out = 'Psalms_105_n.ann'
    old_file = os.path.expanduser('~/Sites/brat/data/coref/Psalms/annotate/Psalms_105.txt') # old file .txt
    mimi_file = os.path.expanduser('~/github/cmerwich/participant-analysis/mimi/mimi-opt/Psalms_105.txt') # mimi file .txt
    text_files = []
    text_files.append(old_file)
    text_files.append(mimi_file)
    translate(new_ann, out, text_files)
    path_a = os.path.expanduser('~/Sites/brat/data/coref/Psalms/annotate/Psalms_105.ann')
    d = CompareAnn(path_a, new_ann) #CompareAnn(path_a, path_b)
    r = minimize(d, w)
    
    if r.success:
        print(f'Max found for w = [{r.x[0]}, {r.x[1]}, {r.x[2]}]')
    else:
        print('No max found')
Optimise([[0.1, 3.0, 3.0]])   

In [ ]:
Stanford:
Sieve 1: Wanneer 1e en 2e persoon coreferen 
Sieve 2: Exact head string match sieve
Entity sieve
3P pronoun aan het einde van MiMi

In [ ]:
Exact head string match sieve

Alle sets aflopen corefs en singletons 
Eerst het lexeem, exacte woord 

Als je strikte match gebruikt, vind je te weinig matches; 
als je alleen head gebruikt (woord in cst) dan vind je te veel. 
Dus strategie is: los met aantal restricties. 

doorzoek alle singletons 
Als je een singleton hebt, dan teruglopen in de tekst, match, dan unite 

mentions die al in een coref zitten, fungeren als potentiële match 

Je loopt vanaf een singleton en corefs terug totdat de eigen tekst positie bereikt (node_tuple gebruiken)

= Als er meerdere matches zijn, de mention matchen die het dichtst bij staat. 
Afstand moet klein zijn. Aanname is hoe dichter de te mentions bij elkaar staan, hoe waarschijnlijker het is 
dat ze naar hetzelfde verwijzen. 
= Afstand meenemen 
= mate van overeenkomst in string 

Doorzoek alle sets van mentions:
Gebruik dan singletons, doorzoek net zo lang totdat de doorzochte mentions qua node niet meer kleiner dan de zoek-mention 

NMPR uitsluiten 
for m in singletons: 
    for coref in coref_lists and singletons: 
        for element in corefs 
        i = 0 
        while i < len(coref) and coref[i] < m.tekst_positie:
            1. Doe match op basis van een aantal criteria:  
            vergelijking doen:
            Tekst afstand meenemen (verschil in node nummer)
            mate van overeenkomst in string
                - getal/geslacht
                - hoofd van de mention, in cst 
                verschillende features combineren om score te berekenen. 
            
            2. Score berekenen voor rang in queu: (score functie)
            bij elke match een score uitrekenen, die score bepaalt de positie in de wachtrij. 
            aan het einde van de zoektocht is er een volle wachtrij, de beste hit wordt vooraan gezet, 
            gebruik dqueu om eerste uit wachtrij te halen
                priority queu, data structuur. Normaal(nqueu (aansluiten), dqueu (bedienen))
            - Als ik klaar ben met het doorzoeken van de singletons, en alle mentions in de wachtrij staan, 
            dan dqueu
            - Dan unite
            - Maak queu leeg
            
        i+=1

In [ ]:
Entity sieve

zelfde procedure als voor de exact string match, 
behalve score functie en de criteria voor de match zijn anders
voorsortering 

twee functies:
    matchfunctie
    waarderingsfunctuie
     
for m in mentions:
    if m.rpt == NMPR:
        1. eerst matchen op bep. criteria 
        - bijv. op hoofd
        - Orthogonale features (features die elkaar niet beinvloeden gebruiken voor vector), 
        vector maken: elke feature is een component van de vector 
        - vector maken van persoon, geslacht, getal. Lengte van vector is mate van overeenkomst score. 
        - Euclidische afstand van vector wordt score. In geval van overeenkomst 0 toekenen, 
        in geval van niet overeenkomst niet 0 toekennen. Elke feature heeft ook een gewicht. 
        Uitzoeken welk gewicht ik per feature toeken
        2, 3, 5
        Euclidische afstand = wortel(sum(elk element uit de vector in het kwadraat))
        lengte = wortel(38)
        2. Score berekenen op basis van die criteria

In [ ]:
Aanpak:
    - eerst kader van beide zeven opzetten in pseudo code
    - eerst werkend krijgen van matchfunctie, waarderingsfunctie

In [2]:
from heapq import heappush, heappop
import math

def heapsort(iterable):
    h = []
    for value in iterable:
         heappush(h, value)
    
    return [heappop(h) for i in range(len(h))]
heapsort([1, 3, 5, 7, 9, 2, 4, 6, 8, 0])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [39]:
l = [2, 3, 5]
i_sum = 0
euc = 0
for i in l:
    i_sqrt = i**2
    i_sum += i_sqrt
    euc = math.sqrt(i_sum)
print(euc)

6.164414002968976


In [40]:
math.sqrt(38)

6.164414002968976

In [7]:
h = []
heappush(h, (5.22, 4, 'write code', 'm'))
heappush(h, (7.23, 5, 'release product', 'm'))
heappush(h, (1.15, 3, 'write spec', 'm'))
heappush(h, (1.99, 2, 'create tests', 'm'))
n = heappop(h)
n

(1.15, 3, 'write spec', 'm')

In [1]:
from functools import lru_cache

from math import exp
from scipy.optimize import minimize

# Dit is een functie waarvan ik weet dat ze een maximum heeft voor
# w = [3.2, 1.7], eens kijken of we dat kunnen vinden.
@lru_cache(maxsize=32)
def f(w):
   print('Called with', w)
   return exp(-(3.2 - w[0])**2 - (1.7 - w[1])**2)

# Omdat de methode naar een minimum zoekt en wij naar een maximum,
# laat ik naar het minimum van het tegengestelde van f() zoeken.

def tegengestelde(w, book, c1, c2):
   return -f(tuple(w))

start_vector = [2, 3]

r = minimize(tegengestelde, start_vector, args = ('Psalms', 23, 34),
	     options={'disp': True})

if r.success:
   print(f'Optimum gevonden voor w = [{r.x[0]}, {r.x[1]}]')
else:
   print('Geen optimum gevonden')

print(f.cache_info())



Called with (2.0, 3.0)
Called with (2.000000014901161, 3.0)
Called with (2.0, 3.000000014901161)
Called with (2.1049227146431804, 2.886333729606122)
Called with (2.1049227295443416, 2.886333729606122)
Called with (2.1049227146431804, 2.886333744507283)
Called with (2.524613573215902, 2.431668648030609)
Called with (2.524613588117063, 2.431668648030609)
Called with (2.524613573215902, 2.43166866293177)
Called with (4.203377007506788, 0.6130083217285573)
Called with (4.203377022407949, 0.6130083217285573)
Called with (4.203377007506788, 0.6130083366297185)
Called with (3.055938426554678, 1.8560667428380198)
Called with (3.0559384414558393, 1.8560667428380198)
Called with (3.055938426554678, 1.856066757739181)
Called with (3.187728733213546, 1.7132939154009559)
Called with (3.1877287481147074, 1.7132939154009559)
Called with (3.187728733213546, 1.713293930302117)
Called with (3.5502245639218533, 1.3205899608587774)
Called with (3.5502245788230145, 1.3205899608587774)
Called with (3.550224

In [2]:
data = [ (1, 7.57), (2, 2.1), (3, 1.2), (4, 2.1), (5, 0.01), 
         (6, 0.5), (7, 0.2), (8, 0.6)]
min(data, key = lambda t: t[1])

(5, 0.01)

In [ ]:
def distance_from_chris(w, book, from_chapter, to_chapter):

    '''
    w = weight_vector = []
    '''
    w_list = []
    r = np.array([0, 0, 0])
    mimi(book, from_chapter, to_chapter, w)
    for c in range(from_chapter, to_chapter +1):
        PATH_A, PATH_B = do_translate(book, c)
        #ps = f'{book}_{c:>03}'
        #mimi_ann = f'{ps}.ann'
        #backup_ann_w = f'{ps}_{w[0]}_{w[1]}_{w[2]}.ann'
        #os.system(f'cp {mimi_ann} {backup_ann_w}')
        
        #print(subprocess.check_output(['md5', f'{PATH_B}']))
        r += CompareAnn(PATH_A, PATH_B)
    print((r[0] + r[2]) / sum(r), 'w: ', w)
    #w_list.append(((r[0] + r[2]) / sum(r), w))
    #get_min(w_list)
    return (r[0] + r[2]) / sum(r) #distance